#### Using the data given 'An Alternative Propulsion Plant for Naval Auxiliary Ships' by Edward Lawrence Stone. (post-graduate thesis) 

First read in the reliability data

In [1]:

import pandas as pd
import openpyxl

data= pd.read_excel("AuxilaryPropulsionPlant_Reliability_Availability_Data.xlsx")
# rel_data = pd.DataFrame({'Component': ['Diesel Engine', 'Clutch', 'Reduction Gear', 'Shaft and Bearings', 'CRP Propellors', 'Fuel Oil Motor', 'Fuel Oil Pump' ],
#                         'MTBF(HRS)': [8000, 50000, 200000, 200000, 25000, 7500, 5500],
#                         'MTTR(HRS)': [8, 'NR', 'NR', 'NR', 15, 18, 4.5]})
print(data)

                Component    MTBF MTTR      R      A
0           Diesel Engine    8000    8  0.999  0.999
1                  Clutch   50000   NR  0.986  1.000
2          Reduction Gear  200000   NR  0.996  1.000
3      Shaft and Bearings  200000   NR  0.996  1.000
4          CRP Propellers   25000   15  0.999  0.999
5          Fuel Oil Motor    7500   18  0.998  0.997
6           Fuel Oil Pump    5500  4.5  0.999  0.999
7       Fuel Oil Purifier   10000    4  1.000  0.999
8          Lube Oil Motor    7500  7.8  0.999  0.998
9           Lube Oil Pump    4000    5  0.999  0.998
10      Jacket Water Pump   27000  7.6  0.999  0.999
11       Fresh Water Pump   12500   12  0.999  0.999
12  Fuel Oil Booster Pump    5500  4.5  0.999  0.999


Next, store the failure and repair rates for each piece of equipment in additional columns

In [ ]:
# adding failure rates to column 3
failure_rates = [1/MTBF for MTBF in data.MTBF]
data.insert(2, 'Failure Rates (fail per HR)', failure_rates)

# add failure rates to column 5 considerign repairable and non-repairable components (MTTR = 'NR') 
repair_rates = [None] * len(data.MTTR)
for i, MTTR in enumerate(data.MTTR): 
    if type(MTTR) is str:
        repair_rates[i] = 'NR'
    else: 
        repair_rates[i] = 1/MTTR
data.insert(4, 'Repair Rates (repairs per HR)', repair_rates)

print(data)

                Component    MTBF  Failure Rates (fail per HR) MTTR  \
0           Diesel Engine    8000                     0.000125    8   
1                  Clutch   50000                     0.000020   NR   
2          Reduction Gear  200000                     0.000005   NR   
3      Shaft and Bearings  200000                     0.000005   NR   
4          CRP Propellers   25000                     0.000040   15   
5          Fuel Oil Motor    7500                     0.000133   18   
6           Fuel Oil Pump    5500                     0.000182  4.5   
7       Fuel Oil Purifier   10000                     0.000100    4   
8          Lube Oil Motor    7500                     0.000133  7.8   
9           Lube Oil Pump    4000                     0.000250    5   
10      Jacket Water Pump   27000                     0.000037  7.6   
11       Fresh Water Pump   12500                     0.000080   12   
12  Fuel Oil Booster Pump    5500                     0.000182  4.5   

   Re

Next, setup a markov chain transition matrix for each component using their failure and repair rates

- Chain consists of two states (failed and working) 

In [ ]:
# loop through each component and create a transition matrix
